# PBL height over the Southern Great Plains

**Be sure to update awsgnssroutils to at least version 1.0.6.**

Import modules. 

In [ ]:
from awsgnssroutils.database import RODatabaseClient
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from netCDF4 import Dataset
from datetime import datetime, timedelta
from scipy.interpolate import interp1d
from scipy.optimize import fmin
from datetime import datetime
import os

gravity = 9.80665

Open port to database. 

In [ ]:
db = RODatabaseClient()

Define region and year

In [ ]:
longituderange = [ -105, -95 ]
latituderange = [ 30, 40 ]
year = 2009
missions = "cosmic1"

In [ ]:
datetimerange = ( datetime(year,1,1).isoformat(), datetime(year+1,1,1).isoformat() )
occs = db.query( missions=missions, datetimerange=datetimerange, 
                longituderange=longituderange, latituderange=latituderange, 
                availablefiletypes="ucar_atmosphericRetrieval" )
print( f'Number of occultations = {occs.size}' )

Download data

In [ ]:
files = occs.download( "romsaf_atmosphericRetrieval" )
localtimes = occs.values( "localtime" )
times = occs.values( "datetime")

In [ ]:
dtimes = []
ltimes = []
pbl_heights = []
fill_value = -1.0e20

z = np.arange( 0, 3000, 10 )

for ifile, file in enumerate( files ): 
    d = Dataset( file, 'r' )
    refractivity = d.variables['refractivity'][:]
    if refractivity.mask.sum() == 0: 
        i = np.arange( refractivity.size )
    else: 
        i = np.argwhere( np.logical_not( refractivity.mask ) ).flatten()
    height = d.variables['geopotential'][i]/gravity
    refractivity = refractivity[i]
    d.close()
    if height.min() > 1000.0: 
        continue
        
    refractivity_gradient = ( refractivity[1:] - refractivity[:-1] ) / ( height[1:] - height[:-1] )
    midheight = 0.5 * ( height[1:] + height[:-1] )
    f = interp1d( midheight, refractivity_gradient, fill_value=fill_value, bounds_error=False, kind='linear' )
    gr = f( z )
    gr = np.ma.masked_where( gr==fill_value, gr )
    pbl_height = z[ np.argmin( gr ) ] / 1000
    timesplit = times[ifile].split("-")
    dtime = datetime( int(timesplit[0]), int(timesplit[1]), int(timesplit[2]), 
                     int(timesplit[3]), int(timesplit[4]) )
    pbl_heights.append( pbl_height )
    dtimes.append( dtime )
    ltimes.append( localtimes[ifile] )
    
pbl_heights = np.array( pbl_heights ).flatten()
ltimes = np.array( ltimes )

### Generate plot

In [ ]:
axeslinewidth = 0.5 
plt.rcParams.update( {
  'font.family': "Times New Roman", 
  'font.size': 8,  
  'font.weight': "normal", 
  'text.usetex': True, 
  'xtick.major.width': axeslinewidth, 
  'xtick.minor.width': axeslinewidth, 
  'ytick.major.width': axeslinewidth, 
  'ytick.minor.width': axeslinewidth, 
  'axes.linewidth': axeslinewidth } ) 

plt.clf()
fig = plt.figure( figsize=(6,4) )
ax = fig.add_axes( [0.10, 0.12, 0.87, 0.80 ] )

ax.set_xlim( 0, 24 )
ax.set_xticks( np.arange(0,24.1,6) )
ax.xaxis.set_minor_locator( MultipleLocator(1) )
ax.set_xlabel( "Local time [hrs]")

ax.set_ylim( 0, 2 )
ax.set_yticks( np.arange(0,3.01,0.5) )
ax.yaxis.set_minor_locator( MultipleLocator(0.1) )
ax.set_ylabel( "PBL height [km]")

ax.scatter( ltimes, pbl_heights, s=0.2, color="k" )

plt.show()